In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_hf.txt"
testDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        test_data = [[] for i in range(num)]
        
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])
                
        with open(testDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                test_data[index].append([content,label])

        for i in range(num):
            # np.random.shuffle(train_data[i])
            train_data[i], test_data[i] = shuffle(train_data[i], test_data[i])
            train_data[i] = np.asarray(train_data[i])
            test_data[i] = np.asarray(test_data[i])

        train_data, test_data = shuffle(train_data, test_data)
        return train_data, test_data
    
    
    train_data = []
    test_data = []
    
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
            
    with open(testDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            test_data.append([content,label])
            
    train_data, test_data = shuffle(train_data, test_data)
    return np.asarray(train_data), np.asarray(test_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, test_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)
        
        te_x = []
        te_y = []
        for ti in test_data:
            x_test, y_test = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            te_x.append(x_test)
            te_y.append(y_test)

        te_x = np.asarray(te_x)
        te_y = np.asarray(te_y)
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(te_x[test_i],te_y[test_i])
            test_acc.append(classifier.train(
                X_train=train_x,
                y_train=train_y,
                X_eval=test_x,
                y_eval=test_y,
                categories=categories,
                epochs=50
            ))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        te_x, te_y = process_file(test_data[:,0], test_data[:,1], word_to_id, num_classes, seq_length)
        # print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(classifier.train(
                X_train=tx[train_i],
                y_train=ty[train_i],
                X_eval=te_x[test_i],
                y_eval=te_y[test_i],
                categories=categories,
                epochs=50
            ))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
train_data, test_data = dataset_split(split_info["expert"])

In [5]:
train_data[3][:3]

array([[list(['who', 'are', 'the', 'cheney', 'with', 'transfer', 'fees', 'of', 'more', 'than', '35,194,003', '?']),
        2],
       [list(['please', 'list', 'the', 'names', 'of', 'all', 'mason', 'with', 'exactly', '54', 'star', 'births', 'in', 'the', 'time', '0223', '.']),
        1],
       [list(['during', 'the', '1307', '-4529', 'season', ',', 'how', 'many', 'mars', 'were', 'transferred', 'in', 'various', 'ice', '?']),
        7]], dtype=object)

In [6]:
test_data[3][:3]

array([[list(['who', 'are', 'the', 'players', 'with', 'transfer', 'fees', 'of', 'more', 'than', '20,000,000', '?']),
        2],
       [list(['please', 'list', 'the', 'names', 'of', 'all', 'neighborhoods', 'with', 'exactly', '56', 'male', 'births', 'in', 'the', 'year', '2017', '.']),
        1],
       [list(['during', 'the', '2010', '-2011', 'season', ',', 'how', 'many', 'players', 'were', 'transferred', 'in', 'various', 'leagues', '?']),
        7]], dtype=object)

In [7]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [8]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [9]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.001)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=3):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [10]:
class ZhouCLSTMModel:
    '''
    Implementation proposal of: https://arxiv.org/pdf/1511.08630
    '''
    def __init__(self, embedding,
        conv_size    = 3,
        conv_filters = 300,
        drop_rate    = 0.5,
        lstm_units   = 150):
        '''Constructor.
        # Parameters:
        conv_size: Size of the convolutions. Number of words that takes each
            convolution step.
        conv_filters: Number of convolution filters.
        drop_rate: Drop rate for the final output of the LSTM layer.
        lstm_units: Size of the states of the LSTM layer.
        '''
        self._embedding    = embedding
        self._conv_size    = conv_size
        self._conv_filters = conv_filters
        self._drop_rate    = drop_rate
        self._lstm_units   = lstm_units

    def __call__(self, input):
        self._embedding_tf = self._create_embedding_layer(
            self._embedding, input)
        
        self._convolution_tf = self._create_convolutional_layers(
            self._conv_size,
            self._conv_filters,
            self._drop_rate,
            self._embedding_tf)
        
        self._lstm_tf = self._create_lstm_layer(
            self._lstm_units,
            self._convolution_tf)
        
        return self._lstm_tf

    def summary(self):
        print('embedding:', str(self._embedding_tf.shape))
        print('conv:', str(self._convolution_tf.shape))
        print('lstm:', str(self._lstm_tf.shape))

    def _create_embedding_layer(self, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return embedded_chars

    def _create_convolutional_layers(self,
        conv_size, num_filters, drop_rate, embedding):
        
        filter_height = conv_size
        filter_width = embedding.shape[2].value

        filter_shape = [filter_height, filter_width, 1, num_filters]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[num_filters]))

        embedding_expanded = tf.expand_dims(embedding, -1)
        conv = tf.nn.conv2d(
            input=embedding_expanded,
            filter=W,
            strides=[1,1,1,1],
            padding='VALID')
        conv_reduced = tf.reshape(
            tensor=conv,
            shape=[-1, conv.shape[1], conv.shape[3]])

        bias = tf.nn.bias_add(conv_reduced, b)
        c = tf.nn.relu(bias)

        d = tf.nn.dropout(c, keep_prob=drop_rate)
        return d

    def _create_lstm_layer(self, lstm_units, conv_input):
        lstm_cell = tf.nn.rnn_cell.LSTMCell(lstm_units)
        sequence = tf.unstack(conv_input, axis=1)
        (_, (h, _)) = tf.nn.static_rnn(lstm_cell, sequence, dtype=tf.float32)

        return h

In [11]:
word_vector = embedding.astype('float32')
model = ZhouCLSTMModel(embedding=word_vector)

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

input: (?, 41)
embedding: (?, 41, 100)
conv: (?, 39, 300)
lstm: (?, 150)
output: (?, 10)


In [ ]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data, test_data = dataset_split(info)
    test_acc_split.append(train_split_data(train_data, test_data, split_type))

random
Fold:  1
Epoch: 1, Train Loss:  1.478, Train Acc: 49.74%, Val Loss: 0.9274, Val Acc: 69.59%, Time: 13.32s *
Epoch: 2, Train Loss:  0.635, Train Acc: 79.67%, Val Loss: 0.6497, Val Acc: 79.42%, Time: 7.09s *
Epoch: 3, Train Loss: 0.3819, Train Acc: 88.12%, Val Loss:  0.529, Val Acc: 83.97%, Time: 7.23s *
Epoch: 4, Train Loss: 0.2568, Train Acc: 91.82%, Val Loss: 0.4969, Val Acc: 84.76%, Time: 7.14s *
Epoch: 5, Train Loss: 0.1837, Train Acc: 93.93%, Val Loss: 0.4654, Val Acc: 86.47%, Time: 7.07s *
Epoch: 6, Train Loss: 0.1362, Train Acc: 95.52%, Val Loss: 0.4723, Val Acc: 87.32%, Time: 7.16s *
Epoch: 7, Train Loss: 0.1082, Train Acc: 96.39%, Val Loss: 0.4307, Val Acc: 88.39%, Time: 7.23s *
Epoch: 8, Train Loss: 0.08477, Train Acc: 96.97%, Val Loss: 0.4779, Val Acc: 88.53%, Time: 7.04s *
Epoch: 9, Train Loss: 0.0663, Train Acc: 97.84%, Val Loss: 0.4789, Val Acc: 88.11%, Time: 7.17s 
Epoch: 10, Train Loss: 0.05785, Train Acc: 97.90%, Val Loss: 0.5038, Val Acc: 88.89%, Time: 7.18s *
E

Epoch: 1, Train Loss:  1.504, Train Acc: 48.52%, Val Loss:  1.003, Val Acc: 69.94%, Time: 8.95s *
Epoch: 2, Train Loss: 0.6452, Train Acc: 79.32%, Val Loss: 0.6398, Val Acc: 80.34%, Time: 7.09s *
Epoch: 3, Train Loss: 0.3927, Train Acc: 87.47%, Val Loss:  0.595, Val Acc: 82.76%, Time: 7.16s *
Epoch: 4, Train Loss: 0.2564, Train Acc: 91.80%, Val Loss: 0.5331, Val Acc: 84.90%, Time: 7.15s *
Epoch: 5, Train Loss: 0.1861, Train Acc: 93.94%, Val Loss: 0.6033, Val Acc: 83.90%, Time: 7.06s 
Epoch: 6, Train Loss: 0.1384, Train Acc: 95.40%, Val Loss: 0.5228, Val Acc: 85.61%, Time: 7.08s *
Epoch: 7, Train Loss: 0.1058, Train Acc: 96.44%, Val Loss: 0.5232, Val Acc: 87.18%, Time: 7.30s *
Epoch: 8, Train Loss: 0.08396, Train Acc: 97.15%, Val Loss: 0.5635, Val Acc: 87.11%, Time: 7.10s 
Epoch: 9, Train Loss: 0.07476, Train Acc: 97.44%, Val Loss: 0.5195, Val Acc: 88.46%, Time: 7.08s *
Epoch: 10, Train Loss:  0.057, Train Acc: 98.10%, Val Loss: 0.6015, Val Acc: 86.89%, Time: 7.23s 
Epoch: 11, Train Los

Epoch: 15, Train Loss: 0.04219, Train Acc: 98.62%, Val Loss:   0.43, Val Acc: 90.81%, Time: 7.15s 
Epoch: 16, Train Loss: 0.03699, Train Acc: 98.80%, Val Loss: 0.5602, Val Acc: 89.74%, Time: 7.13s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.90      0.91      0.91       137
                   Filter       0.86      0.92      0.89       142
    Compute Derived Value       0.71      0.90      0.79       127
            Find Extremum       0.87      0.95      0.91       154
                     Sort       0.95      0.94      0.94       139
          Determine Range       0.85      0.82      0.83       114
Characterize Distribution       0.96      0.83      0.89       150
           Find Anomalies       0.96      0.90      0.93       146
                  Cluster       0.96      0.86      0.91       128
                Correlate       0.95      0.91 

Epoch: 1, Train Loss:  1.477, Train Acc: 50.18%, Val Loss: 0.9054, Val Acc: 70.92%, Time: 9.10s *
Epoch: 2, Train Loss: 0.6355, Train Acc: 79.89%, Val Loss: 0.6009, Val Acc: 81.33%, Time: 7.20s *
Epoch: 3, Train Loss: 0.3852, Train Acc: 87.94%, Val Loss: 0.5818, Val Acc: 82.47%, Time: 7.31s *
Epoch: 4, Train Loss: 0.2505, Train Acc: 91.89%, Val Loss: 0.4548, Val Acc: 87.24%, Time: 7.17s *
Epoch: 5, Train Loss: 0.1855, Train Acc: 94.16%, Val Loss: 0.5747, Val Acc: 84.82%, Time: 7.15s 
Epoch: 6, Train Loss: 0.1353, Train Acc: 95.56%, Val Loss: 0.5078, Val Acc: 87.31%, Time: 7.10s *
Epoch: 7, Train Loss: 0.1045, Train Acc: 96.42%, Val Loss: 0.5345, Val Acc: 86.53%, Time: 7.28s 
Epoch: 8, Train Loss: 0.08732, Train Acc: 97.10%, Val Loss: 0.5595, Val Acc: 87.81%, Time: 7.16s *
Epoch: 9, Train Loss: 0.06672, Train Acc: 97.68%, Val Loss: 0.5558, Val Acc: 87.38%, Time: 7.11s 
Epoch: 10, Train Loss: 0.06244, Train Acc: 97.90%, Val Loss: 0.5638, Val Acc: 86.10%, Time: 7.20s 
Epoch: 11, Train Los

Epoch: 2, Train Loss: 0.7407, Train Acc: 76.47%, Val Loss:   1.72, Val Acc: 56.87%, Time: 7.28s *
Epoch: 3, Train Loss: 0.4542, Train Acc: 85.59%, Val Loss:   1.81, Val Acc: 58.21%, Time: 6.98s *
Epoch: 4, Train Loss: 0.3047, Train Acc: 90.33%, Val Loss:  1.813, Val Acc: 59.14%, Time: 7.10s *
Epoch: 5, Train Loss:  0.205, Train Acc: 93.71%, Val Loss:  2.076, Val Acc: 57.45%, Time: 6.89s 
Epoch: 6, Train Loss: 0.1509, Train Acc: 95.17%, Val Loss:  2.196, Val Acc: 58.50%, Time: 6.89s 
Epoch: 7, Train Loss: 0.1195, Train Acc: 96.31%, Val Loss:  2.402, Val Acc: 58.50%, Time: 7.12s 
Epoch: 8, Train Loss: 0.09255, Train Acc: 96.85%, Val Loss:  2.587, Val Acc: 57.74%, Time: 6.99s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.62      0.45      0.52       179
                   Filter       0.48      0.32      0.38       182
    Compute Derived Value      

...

Epoch: 1, Train Loss:  2.231, Train Acc: 20.70%, Val Loss:  2.159, Val Acc: 24.70%, Time: 9.01s *
Epoch: 2, Train Loss:  1.502, Train Acc: 49.14%, Val Loss:  1.674, Val Acc: 48.21%, Time: 7.16s *
Epoch: 3, Train Loss: 0.9816, Train Acc: 68.07%, Val Loss:  1.481, Val Acc: 58.04%, Time: 7.19s *
Epoch: 4, Train Loss:  0.662, Train Acc: 79.15%, Val Loss:  1.446, Val Acc: 61.26%, Time: 7.18s *
Epoch: 5, Train Loss: 0.4752, Train Acc: 84.91%, Val Loss:  1.562, Val Acc: 63.58%, Time: 7.16s *
Epoch: 6, Train Loss: 0.3555, Train Acc: 88.62%, Val Loss:   1.63, Val Acc: 64.56%, Time: 7.12s *
Epoch: 7, Train Loss: 0.2644, Train Acc: 91.68%, Val Loss:  1.792, Val Acc: 63.72%, Time: 7.16s 
Epoch: 8, Train Loss: 0.2134, Train Acc: 93.21%, Val Loss:  1.974, Val Acc: 62.88%, Time: 7.24s 
Epoch: 9, Train Loss: 0.1649, Train Acc: 94.61%, Val Loss:  1.905, Val Acc: 66.11%, Time: 7.15s *
Epoch: 10, Train Loss: 0.1271, Train Acc: 96.02%, Val Loss:  2.099, Val Acc: 65.26%, Time: 7.10s 
Epoch: 11, Train Loss:

Epoch: 1, Train Loss:  2.271, Train Acc: 19.07%, Val Loss:  2.003, Val Acc: 29.83%, Time: 8.87s *
Epoch: 2, Train Loss:  1.571, Train Acc: 46.07%, Val Loss:  1.608, Val Acc: 44.28%, Time: 7.17s *
Epoch: 3, Train Loss:  1.025, Train Acc: 67.44%, Val Loss:  1.476, Val Acc: 53.72%, Time: 7.17s *
Epoch: 4, Train Loss: 0.7001, Train Acc: 78.37%, Val Loss:  1.392, Val Acc: 58.58%, Time: 7.06s *
Epoch: 5, Train Loss: 0.5114, Train Acc: 84.54%, Val Loss:  1.453, Val Acc: 60.94%, Time: 7.03s *
Epoch: 6, Train Loss: 0.3945, Train Acc: 87.64%, Val Loss:  1.508, Val Acc: 62.09%, Time: 7.08s *
Epoch: 7, Train Loss: 0.2866, Train Acc: 91.19%, Val Loss:  1.632, Val Acc: 64.88%, Time: 7.12s *
Epoch: 8, Train Loss: 0.2147, Train Acc: 93.34%, Val Loss:  1.809, Val Acc: 63.38%, Time: 7.07s 
Epoch: 9, Train Loss: 0.1763, Train Acc: 94.45%, Val Loss:   2.14, Val Acc: 61.73%, Time: 7.05s 
Epoch: 10, Train Loss: 0.1388, Train Acc: 95.58%, Val Loss:  2.345, Val Acc: 62.16%, Time: 7.21s 
Epoch: 11, Train Loss:

Epoch: 4, Train Loss: 0.3619, Train Acc: 88.64%, Val Loss:  1.555, Val Acc: 66.39%, Time: 7.21s *
Epoch: 5, Train Loss: 0.2585, Train Acc: 91.89%, Val Loss:  1.428, Val Acc: 66.13%, Time: 7.27s 
Epoch: 6, Train Loss: 0.1877, Train Acc: 93.80%, Val Loss:  1.589, Val Acc: 65.63%, Time: 7.10s 
Epoch: 7, Train Loss: 0.1401, Train Acc: 95.58%, Val Loss:  1.669, Val Acc: 66.55%, Time: 7.18s *
Epoch: 8, Train Loss: 0.1138, Train Acc: 96.23%, Val Loss:  1.966, Val Acc: 65.88%, Time: 7.16s 
Epoch: 9, Train Loss: 0.09633, Train Acc: 96.81%, Val Loss:  1.864, Val Acc: 65.55%, Time: 7.19s 
Epoch: 10, Train Loss: 0.07511, Train Acc: 97.52%, Val Loss:  2.305, Val Acc: 63.11%, Time: 7.28s 
Epoch: 11, Train Loss: 0.06145, Train Acc: 97.99%, Val Loss:  2.246, Val Acc: 63.61%, Time: 7.25s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.45      0.22      0.29        8

Epoch: 13, Train Loss: 0.05056, Train Acc: 98.31%, Val Loss:  2.182, Val Acc: 64.10%, Time: 7.18s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.34      0.46      0.39       100
                   Filter       0.61      0.45      0.52       100
    Compute Derived Value       0.47      0.47      0.47       114
            Find Extremum       0.82      0.87      0.84       147
                     Sort       0.74      1.00      0.85        90
          Determine Range       0.77      0.52      0.62        92
Characterize Distribution       0.63      0.78      0.70       115
           Find Anomalies       0.58      0.57      0.58        89
                  Cluster       0.99      0.70      0.82       125
                Correlate       0.74      0.67      0.70       156

                micro avg       0.66      0.66      0.66      1128
           

Fold:  8
Epoch: 1, Train Loss:  1.706, Train Acc: 41.98%, Val Loss:  1.541, Val Acc: 52.27%, Time: 9.48s *
Epoch: 2, Train Loss:  0.876, Train Acc: 72.49%, Val Loss:  1.321, Val Acc: 63.92%, Time: 7.47s *
Epoch: 3, Train Loss: 0.5478, Train Acc: 82.97%, Val Loss:  1.256, Val Acc: 68.61%, Time: 7.50s *
Epoch: 4, Train Loss: 0.3842, Train Acc: 88.20%, Val Loss:  1.266, Val Acc: 70.88%, Time: 7.45s *
Epoch: 5, Train Loss: 0.2758, Train Acc: 91.56%, Val Loss:  1.506, Val Acc: 69.60%, Time: 7.51s 
Epoch: 6, Train Loss: 0.1999, Train Acc: 93.61%, Val Loss:  1.503, Val Acc: 69.46%, Time: 7.44s 
Epoch: 7, Train Loss: 0.1525, Train Acc: 95.32%, Val Loss:  1.389, Val Acc: 73.01%, Time: 7.49s *
Epoch: 8, Train Loss: 0.1276, Train Acc: 95.93%, Val Loss:  1.473, Val Acc: 70.74%, Time: 7.46s 
Epoch: 9, Train Loss: 0.1056, Train Acc: 96.57%, Val Loss:  1.558, Val Acc: 72.59%, Time: 7.54s 
Epoch: 10, Train Loss: 0.07479, Train Acc: 97.53%, Val Loss:  1.571, Val Acc: 73.30%, Time: 7.40s *
Epoch: 11, Tr